# Enterprise Government Document Processing with A2A

## Capstone Project Demo

This notebook demonstrates:
- Multi-agent orchestration using Google ADK
- Agent2Agent (A2A) protocol for cross-organizational integration
- Security controls with PII filtering at organizational boundaries
- Real-world government document processing use case

## Setup

In [ ]:
import os
import logging
from pathlib import Path
from dotenv import load_dotenv

# Load environment
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ Environment loaded")

## Step 1: Verify API Key

In [ ]:
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key or api_key == "your_api_key_here":
    print("⚠️  GOOGLE_API_KEY not configured!")
    print("Please set your API key in .env file")
    print("Get your key at: https://aistudio.google.com/app/apikey")
else:
    print("✓ API key configured")

## Step 2: Start Vendor Server (Optional)

In a separate terminal, run:
```bash
python -m vendor.vendor_server
```

This starts the A2A vendor service at `http://localhost:8001`

## Step 3: Test Vendor Connectivity

In [ ]:
from tools.vendor_connector import test_vendor_connection

vendor_status = test_vendor_connection()
print(f"Vendor Status: {vendor_status['status']}")

if vendor_status['status'] == 'success':
    print("✓ Vendor server is reachable")
    print(f"  Agent Card: {vendor_status.get('agent_card_url')}")
else:
    print("⚠️  Vendor not reachable (will simulate in demo)")

## Step 4: Load Sample Document

In [ ]:
sample_doc = Path("samples/sample_document.txt")

if sample_doc.exists():
    with open(sample_doc, 'r', encoding='utf-8') as f:
        doc_content = f.read()
    
    print(f"✓ Loaded: {sample_doc.name}")
    print(f"  Size: {len(doc_content)} characters")
    print(f"  Preview: {doc_content[:200]}...")
else:
    print(f"⚠️  Document not found: {sample_doc}")

## Step 5: Create Intake Agent

In [ ]:
from agents import create_intake_agent

intake_agent = create_intake_agent()

print(f"✓ Created: {intake_agent.name}")
print(f"  Description: {intake_agent.description}")
print(f"  Tools: {[tool.__name__ for tool in intake_agent.tools]}")

## Step 6: Run Document Intake

In [ ]:
from adk.runners import Runner

intake_runner = Runner(agent=intake_agent)
intake_prompt = f"Please validate the document at: {sample_doc}"

print("Running Intake Agent...")
intake_result = await intake_runner.run(intake_prompt)

print("\n--- Intake Agent Response ---")
print(intake_result.response_text)

## Step 7: Create Processing Agent with A2A

In [ ]:
from agents import create_processing_agent

processing_agent = create_processing_agent()

print(f"✓ Created: {processing_agent.name}")
print(f"  Tools: {[tool.__name__ for tool in processing_agent.tools]}")
print(f"  Sub-agents: {[agent.name for agent in processing_agent.sub_agents]}")
print("\n  A2A Integration:")
print(f"    → {processing_agent.sub_agents[0].name} (RemoteA2aAgent)")

## Step 8: Run Complete Processing Pipeline

This demonstrates:
1. OCR text extraction
2. PII filtering (pre-vendor)
3. **A2A vendor call** (cross-organizational boundary)
4. Security verification (post-vendor)
5. Final result compilation

In [ ]:
processing_runner = Runner(agent=processing_agent)

processing_prompt = f"""Process the document at {sample_doc} through the complete pipeline:

1. Extract text using OCR
2. Apply security filtering (mask PII)
3. Send to external vendor via A2A for translation (Spanish to English)
4. Verify vendor response
5. Return final processed document

Document type: birth_certificate
Target language: English"""

print("Running Processing Agent with A2A integration...")
print("  → OCR extraction")
print("  → PII filtering")
print("  → A2A vendor call [BOUNDARY]")
print("  → Security verification")
print("  → Final compilation\n")

processing_result = await processing_runner.run(processing_prompt)

print("\n--- Processing Agent Response ---")
print(processing_result.response_text)

## Step 9: Inspect PII Filtering

In [ ]:
from security import security_filter

# Test PII detection
filter_result = security_filter(doc_content, mode="detect")

print("PII Detection Results:")
print(f"  Total PII found: {filter_result['pii_count']}")
print(f"  Categories: {list(filter_result['detected_pii'].keys())}")

for pii_type, instances in filter_result['detected_pii'].items():
    print(f"\n  {pii_type}:")
    for instance in instances[:3]:  # Show first 3
        print(f"    - {instance}")

## Step 10: Test PII Masking

In [ ]:
mask_result = security_filter(doc_content, mode="mask")

print("PII Masking Results:")
print(f"  Status: {mask_result['status']}")
print(f"  Masked count: {mask_result['masked_count']}")
print(f"  Categories masked: {list(mask_result['pii_summary'].keys())}")

print("\nMasked Text Preview:")
print(mask_result['filtered_text'][:500] + "...")

## Summary

### Capstone Objectives Demonstrated

✓ **Multi-agent orchestration**: IntakeAgent → ProcessingAgent workflow  
✓ **A2A protocol**: RemoteA2aAgent for cross-organizational integration  
✓ **Security boundaries**: PII filtering at organizational boundaries  
✓ **Tool integration**: OCR, security_filter, vendor translation  
✓ **Sub-agent delegation**: ProcessingAgent delegates to RemoteA2aAgent  
✓ **Real-world use case**: Government document translation service  

### Architecture Highlights

```
Government Ministry (Internal)          External Vendor
┌─────────────────────────┐            ┌──────────────┐
│  IntakeAgent            │            │              │
│    ↓                    │            │              │
│  ProcessingAgent        │            │              │
│    - OCR Tool           │            │              │
│    - Security Filter    │   A2A      │  Docs        │
│    - RemoteA2aAgent ----├───────────→│  Translator  │
│      (Consumer)         │            │  (Provider)  │
└─────────────────────────┘            └──────────────┘
     [PII Filtered]                     [to_a2a()]
```

### Key Technologies

- **Google ADK**: Agent framework
- **Agent2Agent (A2A)**: Cross-org protocol
- **Gemini 2.0 Flash Lite**: LLM model
- **FastAPI/Uvicorn**: A2A server
- **RemoteA2aAgent**: A2A client
- **to_a2a()**: A2A server creation